In [1]:
n = 2

a = 3
b = 0

In [2]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

In [3]:
reg_A = QuantumRegister(n, name='a')
reg_B = QuantumRegister(n, name='b')
reg_C = QuantumRegister(1, name='c')

In [4]:
qc = QuantumCircuit(reg_B, reg_A, reg_C)

# 1
for i in range(1, n): # 1..n-1
    qc.cx(reg_A[i], reg_B[i])

# 2
qc.cx(reg_A[n-1], reg_C)
for i in range(n-2, 0, -1): # n-2..1
    qc.cx(reg_A[i], reg_A[i+1])

# 3    
for i in range(0, n-1):
    qc.ccx(reg_A[i], reg_B[i], reg_A[i+1])
qc.ccx(reg_A[n-1], reg_B[n-1], reg_C)

# 4
for i in range(n-1, 0, -1):
    qc.cx(reg_A[i], reg_B[i])
    qc.ccx(reg_A[i-1], reg_B[i-1], reg_A[i])

# 5
for i in range(1, n-1):
    qc.cx(reg_A[i], reg_A[i+1])
    
# 6
for i in range(0, n):
    qc.cx(reg_A[i], reg_B[i])

In [5]:
# qc.draw(output='mpl')

In [6]:
cr0 = ClassicalRegister(n, name='cb')
cr1 = ClassicalRegister(n, name='ca')
cr2 = ClassicalRegister(1, name='carry')

qc_meas = QuantumCircuit(reg_B, reg_A, reg_C, cr0, cr1, cr2)
qc_meas.barrier(reg_B[:], reg_A[:], reg_C[:])
qc_meas.measure(reg_B[:], cr0[:])
qc_meas.measure(reg_A[:], cr1[:])
qc_meas.measure(reg_C[:], cr2[:])

# qc_meas.draw(output='mpl')

In [7]:
qc_init = QuantumCircuit(reg_B, reg_A)

b_bits = [ int(x) for x in '{:0{size}b}'.format(b, size=n) ]
a_bits = [ int(x) for x in '{:0{size}b}'.format(a, size=n) ]

for i in range(n):
    if a_bits[n-(i+1)] == 1:
        qc_init.x(reg_A[i])
        
for i in range(n):
    if b_bits[n-(i+1)] == 1:
        qc_init.x(reg_B[i])

qc_init.barrier(reg_B[:], reg_A[:])
        
# qc_init.draw(output='mpl')

In [8]:
qc_all = qc_init + qc + qc_meas
# qc_all.draw(output='mpl')

<ipython-input-8-7badd2fe3392>:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  qc_all = qc_init + qc + qc_meas
/home/bartek/MGR/masterThesisProject/venv/lib/python3.8/site-packages/qiskit/circuit/quantumcircuit.py:869: DeprecationWarning: The QuantumCircuit.combine() method is being deprecated. Use the compose() method which is more flexible w.r.t circuit register compatibility.
  return self.combine(rhs)


In [9]:
from qiskit import Aer
from qiskit import execute

backend_sim = Aer.get_backend('qasm_simulator')
job_sim = execute(qc_all, backend_sim, shots=1024)

result_sim = job_sim.result()

In [10]:
counts = result_sim.get_counts(qc_all)
print(counts)

{'0 11 11': 1024}
